In [ ]:
# Pie a mano con porcentajes de hatos

In [22]:
import plotly.graph_objects as go
import random

# Datos de la distribución
labels = ['No DE FINCAS 1 A 50', 'No DE FINCAS 51 A 100', 'No DE FINCAS 101 A 500', 'No DE FINCAS 501 O MAS']
values = [74.3, 14.1, 10.7, 0.7]

# Generar colores aleatorios para cada segmento
colors = [f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})' for _ in labels]

# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='percent', marker=dict(colors=colors))])
fig.update_layout(title='Distribución de Fincas por Cantidad de Bovinos')

# Mostrar el gráfico
fig.show()


# Tablas y gráficos genéricos a partir de archivos .csv en la carpeta de trabajo

V. 0.4 (NO FUNCIONAL)

In [17]:
# V 0.4

import os
import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import plotly.graph_objects as go

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    display(HTML("<p style='color:red'>No se encontraron archivos CSV en el directorio especificado.</p>"))
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        display(HTML(f"<p style='color:red'>El archivo {file} está vacío y no se puede cargar.</p>"))

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    display(HTML("<p style='color:red'>No se pudo cargar ningún archivo CSV con datos.</p>"))
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Mostrar los campos disponibles
fields = data.columns.tolist()
display(HTML("<h3>Campos disponibles:</h3>"))
display(HTML(f"<p>{', '.join(fields)}</p>"))

# Crear los dropdown menus
index_dropdown = widgets.SelectMultiple(
    options=fields,
    description='Índice:',
    rows=5,
    layout={'width': 'auto'}
)

variable_dropdown = widgets.SelectMultiple(
    options=fields,
    description='Variable:',
    rows=5,
    layout={'width': 'auto'}
)

x_axis_dropdown = widgets.SelectMultiple(
    options=fields,
    description='Eje X:',
    rows=5,
    layout={'width': 'auto'}
)

y_axis_dropdown = widgets.SelectMultiple(
    options=fields,
    description='Eje Y:',
    rows=5,
    layout={'width': 'auto'}
)

chart_type_dropdown = widgets.Dropdown(
    options=['bar', 'line', 'scatter'],
    description='Tipo de Gráfico:',
    layout={'width': 'auto'}
)

departamento_dropdown = widgets.SelectMultiple(
    options=data['DEPARTAMENTO'].unique(),
    description='Departamento:',
    rows=5,
    layout={'width': 'auto'}
)

municipio_dropdown = widgets.SelectMultiple(
    options=data['MUNICIPIO'].unique(),
    description='Municipio:',
    rows=5,
    layout={'width': 'auto'}
)

# Crear las salidas HTML para la tabla y el gráfico
table_output = widgets.Output()
chart_output = widgets.Output()

# Generar y mostrar la tabla
def generate_output(change):
    selected_departamentos = departamento_dropdown.value
    selected_municipios = municipio_dropdown.value
    selected_indexes = index_dropdown.value
    selected_variables = variable_dropdown.value
    selected_x_fields = x_axis_dropdown.value
    selected_y_fields = y_axis_dropdown.value
    chart_type = chart_type_dropdown.value

    # Filtrar los datos basado en las selecciones realizadas
    filtered_data = data[data["DEPARTAMENTO"].isin(selected_departamentos) & data["MUNICIPIO"].isin(selected_municipios)]

    # Verificar si los índices seleccionados existen en el DataFrame filtrado
    selected_indexes = [idx for idx in selected_indexes if idx in filtered_data.columns]

    if len(selected_indexes) > 0:
        filtered_data = filtered_data[selected_indexes]
    else:
        filtered_data = filtered_data[fields]  # Usar todos los campos del DataFrame

    filtered_data = filtered_data[selected_variables]
    filtered_data = filtered_data[selected_x_fields]
    filtered_data = filtered_data[selected_y_fields]

    # Limpiar las salidas HTML
    table_output.clear_output()
    chart_output.clear_output()

    # Generar y mostrar la tabla
    with table_output:
        display(HTML("<h2>Tabla</h2>"))
        display(filtered_data)

        # Generar el contenido de la tabla en formato markdown
        markdown_content = filtered_data.to_markdown()

        # Crear el widget de Textarea para mostrar el contenido en formato markdown
        textarea_widget = widgets.Textarea(value=markdown_content, layout={'height': '100%', 'width': '100%'})

        # Crear la función para crear un enlace de descarga
        def create_download_link(content, filename):
            buffer = io.BytesIO(content.encode())
            download_link = widgets.Button(description="Descargar", layout={'width': 'auto'})
            download_link.style.button_color = 'lightgreen'
            download_link.style.font_weight = 'bold'

            def download_table(_):
                with open(filename, 'wb') as f:
                    f.write(buffer.getbuffer())

            download_link.on_click(download_table)
            return download_link

        # Mostrar el widget de Textarea y el enlace de descarga
        display(textarea_widget)
        display(create_download_link(markdown_content, 'tabla.md'))

    # Generar y mostrar el gráfico
    with chart_output:
        display(HTML("<h2>Gráfico</h2>"))
        fig = go.Figure()

        for y_field in selected_y_fields:
            fig.add_trace(go.Scatter(
                x=filtered_data[selected_x_fields],
                y=filtered_data[y_field],
                mode='lines',
                name=y_field
            ))

        fig.update_layout(title='Gráfico', xaxis_title='Eje X', yaxis_title='Eje Y', barmode='group')
        fig.show()

# Registrar la función de generación de la tabla y el gráfico para los eventos de cambio de los dropdowns
departamento_dropdown.observe(generate_output, names='value')
municipio_dropdown.observe(generate_output, names='value')
index_dropdown.observe(generate_output, names='value')
variable_dropdown.observe(generate_output, names='value')
x_axis_dropdown.observe(generate_output, names='value')
y_axis_dropdown.observe(generate_output, names='value')
chart_type_dropdown.observe(generate_output, names='value')

# Mostrar los dropdowns y las salidas HTML
display(widgets.HBox([departamento_dropdown, municipio_dropdown]))
display(widgets.HBox([index_dropdown, variable_dropdown]))
display(widgets.HBox([x_axis_dropdown, y_axis_dropdown]))
display(widgets.HBox([chart_type_dropdown]))
display(widgets.VBox([table_output, chart_output]))

print(df.columns)



Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO',
       'BUCERROS < 1 AÑO', 'HEMBRAS 1 - 2 AÑOS', 'MACHOS 1 - 2 AÑOS',
       'HEMBRAS 2 - 3 AÑOS', 'MACHOS 2 - 3 AÑOS', 'HEMBRAS > 3 AÑOS',
       'MACHOS > 3 AÑOS', 'TOTAL BUFALOS - 2022', ' No DE FINCAS 1 A 50',
       ' No DE FINCAS 51 A 100', ' No DE FINCAS 101 A 500',
       ' No DE FINCAS 501 O MAS', 'TOTAL FINCAS CON BUFALOS - 2022'],
      dtype='object')


KeyError: (' No DE FINCAS 501 O MAS',)

# Variante para generar una tabla y pie específico para predios

In [21]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown
import plotly.graph_objects as go

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Filtrar y mostrar la tabla deseada
def filter_table(departamento, municipio):
    filtered_data = data.loc[(data['DEPARTAMENTO'].isin(departamento)) & (data['MUNICIPIO'].isin(municipio))]
    filtered_data = filtered_data[['MUNICIPIO', 'TOTAL BOVINOS', ' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100',
                                   ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS']]
    total_row = filtered_data.sum().to_frame().T
    filtered_data = pd.concat([filtered_data, total_row], ignore_index=True)
    display(Markdown(filtered_data.to_markdown()))

    # Generar la gráfica de pastel
    labels = ['No DE FINCAS 1 A 50', 'No DE FINCAS 51 A 100', 'No DE FINCAS 101 A 500', 'No DE FINCAS 501 O MAS']
    values = total_row[[' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100', ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS']]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
    fig.update_layout(title='Distribución de Fincas por Cantidad de Bovinos')
    fig.show()

# Obtener la lista de departamentos y municipios únicos
departamentos = data['DEPARTAMENTO'].unique().tolist()
municipios = data['MUNICIPIO'].unique().tolist()

# Crear los campos de selección múltiple para departamentos y municipios
departamento_dropdown = widgets.SelectMultiple(options=departamentos, description='Departamento:')
municipio_dropdown = widgets.SelectMultiple(options=municipios, description='Municipio:')

# Función de actualización del campo de selección de municipios basado en la selección de departamentos
def update_municipios(*args):
    selected_departamentos = departamento_dropdown.value
    filtered_municipios = data.loc[data['DEPARTAMENTO'].isin(selected_departamentos), 'MUNICIPIO'].unique().tolist()
    municipio_dropdown.options = filtered_municipios

# Asociar la función de actualización al evento de cambio de selección de departamentos
departamento_dropdown.observe(update_municipios, 'value')

# Mostrar los campos de selección
widgets.interact(filter_table, departamento=departamento_dropdown, municipio=municipio_dropdown)


interactive(children=(SelectMultiple(description='Departamento:', options=('ANTIOQUIA', 'ATLANTICO', 'BOGOTA D…

<function __main__.filter_table(departamento, municipio)>

# Tablas y gráficos genéricos a partir de archivos .csv en la carpeta de trabajo

V. 0.3

In [4]:
# V. 0.3

import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import io

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Mostrar los campos disponibles
fields = data.columns.tolist()
print("Campos disponibles:")
print(fields)

# Crear los dropdown menus para seleccionar los campos
index_dropdown = widgets.Dropdown(options=fields, description="Index:")
variable_dropdown = widgets.Dropdown(options=fields, description="Variable:")
x_axis_dropdown = widgets.Dropdown(options=fields, description="Eje X:")
y_axis_dropdown = widgets.Dropdown(options=fields, description="Eje Y:")
chart_type_dropdown = widgets.Dropdown(options=["bar", "line", "scatter"], description="Tipo de gráfico:")

# Filtrar el campo "DEPARTAMENTO" para obtener las opciones del dropdown de MUNICIPIO
departamento_options = data["DEPARTAMENTO"].unique()

# Crear el dropdown menu para seleccionar el campo "DEPARTAMENTO"
departamento_dropdown = widgets.Dropdown(options=departamento_options, description="Departamento:")

# Crear el dropdown menu para seleccionar el campo "MUNICIPIO" basado en el filtro del campo "DEPARTAMENTO"
municipio_dropdown = widgets.SelectMultiple(description="Municipio:")

# Función para actualizar las opciones del dropdown de MUNICIPIO basado en el filtro del dropdown de DEPARTAMENTO
def update_municipio_options(change):
    selected_departamentos = [departamento_dropdown.value]
    municipio_options = data[data["DEPARTAMENTO"].isin(selected_departamentos)]["MUNICIPIO"].unique()
    municipio_dropdown.options = municipio_options

# Asignar la función de actualización de opciones al evento "value" del dropdown de DEPARTAMENTO
departamento_dropdown.observe(update_municipio_options, 'value')

# Crear las salidas HTML para la tabla y el gráfico
table_output = widgets.Output()
chart_output = widgets.Output()

# Función para generar y mostrar la tabla y el gráfico seleccionados
def generate_output(change):
    # Obtener las selecciones realizadas
    selected_departamentos = [departamento_dropdown.value]
    selected_municipios = list(municipio_dropdown.value)
    
    # Obtener los campos seleccionados
    index_field = index_dropdown.value
    variable_field = variable_dropdown.value
    x_field = x_axis_dropdown.value
    y_field = y_axis_dropdown.value
    chart_type = chart_type_dropdown.value
    
    # Filtrar los datos basado en las selecciones realizadas
    filtered_data = data[data["DEPARTAMENTO"].isin(selected_departamentos) & data["MUNICIPIO"].isin(selected_municipios)]
    
    # Limpiar las salidas HTML
    table_output.clear_output()
    chart_output.clear_output()
    
    # Generar y mostrar la tabla
    with table_output:
        display(HTML("<h2>Tabla</h2>"))
        display(filtered_data)
        
        # Generar el contenido de la tabla en formato markdown
        markdown_content = filtered_data.to_markdown()
        
        # Crear el widget de Textarea para mostrar el contenido en formato markdown
        textarea_widget = widgets.Textarea(value=markdown_content, layout={'height': '100%', 'width': '100%'})
        
        # Crear la función para crear un enlace de descarga
        def create_download_link(content, filename):
            buffer = io.BytesIO(content.encode())
            download_link = widgets.Button(description="Descargar", layout={'width': 'auto'})
            download_link.style.button_color = 'lightgreen'
            download_link.style.font_weight = 'bold'

            def download_table(_):
                with open(filename, 'wb') as f:
                    f.write(buffer.getbuffer())

            download_link.on_click(download_table)
            return download_link
        
        # Mostrar el widget de Textarea
        display(textarea_widget)
        
        # Crear y mostrar el enlace de descarga
        download_link = create_download_link(markdown_content, filename="tabla.md")
        display(download_link)
    
    # Generar y mostrar el gráfico
    with chart_output:
        try:
            fig = go.Figure()
            
            if chart_type == "bar":
                fig.add_trace(go.Bar(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "line":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="lines",
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "scatter":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="markers",
                    name=f"{y_field} ({x_field})"
                ))

            fig.update_layout(
                title=f"Gráfico de {chart_type} de {y_field} por {x_field}",
                xaxis=dict(title=x_field),
                yaxis=dict(title=y_field)
            )

            fig.show()
        except ValueError as e:
            print(f"No se pudo generar la gráfica. Error: {str(e)}")

# Asignar la función de generación de tabla y gráfico al evento "change" de los dropdown menus
index_dropdown.observe(generate_output, 'value')
variable_dropdown.observe(generate_output, 'value')
x_axis_dropdown.observe(generate_output, 'value')
y_axis_dropdown.observe(generate_output, 'value')
chart_type_dropdown.observe(generate_output, 'value')
departamento_dropdown.observe(generate_output, 'value')
municipio_dropdown.observe(generate_output, 'value')

# Mostrar los dropdown menus y las salidas HTML
display(index_dropdown, variable_dropdown, x_axis_dropdown, y_axis_dropdown, chart_type_dropdown)
display(departamento_dropdown, municipio_dropdown)
display(table_output)
display(chart_output)


# Tablas y gráficos genéricos a partir de archivos .csv en la carpeta de trabajo

V. 0.2

In [3]:
# V 0.2

import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import io

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Mostrar los campos disponibles
fields = data.columns.tolist()
print("Campos disponibles:")
print(fields)

# Crear los dropdown menus para seleccionar los campos
index_dropdown = widgets.Dropdown(options=fields, description="Index:")
variable_dropdown = widgets.Dropdown(options=fields, description="Variable:")
x_axis_dropdown = widgets.Dropdown(options=fields, description="Eje X:")
y_axis_dropdown = widgets.Dropdown(options=fields, description="Eje Y:")
chart_type_dropdown = widgets.Dropdown(options=["bar", "line", "scatter"], description="Tipo de gráfico:")

# Filtrar el campo "DEPARTAMENTO" para obtener las opciones del dropdown de MUNICIPIO
departamento_options = data["DEPARTAMENTO"].unique()

# Crear el dropdown menu para seleccionar el campo "DEPARTAMENTO"
departamento_dropdown = widgets.Dropdown(options=departamento_options, description="Departamento:")

# Crear el dropdown menu para seleccionar el campo "MUNICIPIO" basado en el filtro del campo "DEPARTAMENTO"
municipio_dropdown = widgets.SelectMultiple(description="Municipio:")

# Función para actualizar las opciones del dropdown de MUNICIPIO basado en el filtro del dropdown de DEPARTAMENTO
def update_municipio_options(change):
    selected_departamentos = [departamento_dropdown.value]
    municipio_options = data[data["DEPARTAMENTO"].isin(selected_departamentos)]["MUNICIPIO"].unique()
    municipio_dropdown.options = municipio_options

# Asignar la función de actualización de opciones al evento "value" del dropdown de DEPARTAMENTO
departamento_dropdown.observe(update_municipio_options, 'value')

# Crear las salidas HTML para la tabla y el gráfico
table_output = widgets.Output()
chart_output = widgets.Output()

# Función para generar y mostrar la tabla y el gráfico seleccionados
def generate_output(change):
    # Obtener las selecciones realizadas
    selected_departamentos = [departamento_dropdown.value]
    selected_municipios = list(municipio_dropdown.value)
    
    # Obtener los campos seleccionados
    index_field = index_dropdown.value
    variable_field = variable_dropdown.value
    x_field = x_axis_dropdown.value
    y_field = y_axis_dropdown.value
    chart_type = chart_type_dropdown.value
    
    # Filtrar los datos basado en las selecciones realizadas
    filtered_data = data[data["DEPARTAMENTO"].isin(selected_departamentos) & data["MUNICIPIO"].isin(selected_municipios)]
    
    # Limpiar las salidas HTML
    table_output.clear_output()
    chart_output.clear_output()
    
    # Generar y mostrar la tabla
    with table_output:
        display(HTML("<h2>Tabla</h2>"))
        display(filtered_data)
        
        # Generar el contenido de la tabla en formato markdown
        markdown_content = filtered_data.to_markdown()
        
        # Crear el botón de descarga de la tabla en formato markdown
        download_button = widgets.Button(description="Descargar Tabla en Markdown")
        
        # Función para descargar la tabla en formato markdown
        def download_table(_):
            # Crear un archivo en memoria para almacenar el contenido en formato markdown
            buffer = io.StringIO(markdown_content)
            
            # Descargar el archivo
            filename = "tabla.md"
            widget_value = widgets.Textarea(value=buffer.getvalue(), layout={'height': '100%', 'width': '100%'})
            display(widget_value)
            widget_value.download_link(filename=filename, mimetype='text/markdown', text='Descargar')
        
        # Asignar la función de descarga al evento "click" del botón
        download_button.on_click(download_table)
        
        # Mostrar el botón de descarga
        display(download_button)
    
    # Generar y mostrar el gráfico
    with chart_output:
        try:
            fig = go.Figure()
            
            if chart_type == "bar":
                fig.add_trace(go.Bar(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "line":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="lines",
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "scatter":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="markers",
                    name=f"{y_field} ({x_field})"
                ))

            fig.update_layout(
                title=f"Gráfico de {chart_type} de {y_field} por {x_field}",
                xaxis=dict(title=x_field),
                yaxis=dict(title=y_field)
            )

            fig.show()
        except ValueError as e:
            print(f"No se pudo generar la gráfica. Error: {str(e)}")

# Asignar la función de generación de tabla y gráfico al evento "change" de los dropdown menus
index_dropdown.observe(generate_output, 'value')
variable_dropdown.observe(generate_output, 'value')
x_axis_dropdown.observe(generate_output, 'value')
y_axis_dropdown.observe(generate_output, 'value')
chart_type_dropdown.observe(generate_output, 'value')
departamento_dropdown.observe(generate_output, 'value')
municipio_dropdown.observe(generate_output, 'value')

# Mostrar los dropdown menus y las salidas HTML
display(index_dropdown, variable_dropdown, x_axis_dropdown, y_axis_dropdown, chart_type_dropdown)
display(departamento_dropdown, municipio_dropdown)
display(table_output)
display(chart_output)


Campos disponibles:
['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', 'BUCERROS < 1 AÑO', 'HEMBRAS 1 - 2 AÑOS', 'MACHOS 1 - 2 AÑOS', 'HEMBRAS 2 - 3 AÑOS', 'MACHOS 2 - 3 AÑOS', 'HEMBRAS > 3 AÑOS', 'MACHOS > 3 AÑOS', 'TOTAL BUFALOS - 2022', ' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100', ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS', 'TOTAL FINCAS CON BUFALOS - 2022', 'TERNERAS < 1 AÑO', 'TERNEROS < 1 AÑO', 'TOTAL BOVINOS', 'TOTAL FINCAS CON BOVINOS']


Dropdown(description='Index:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', '…

Dropdown(description='Variable:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO'…

Dropdown(description='Eje X:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', '…

Dropdown(description='Eje Y:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', '…

Dropdown(description='Tipo de gráfico:', options=('bar', 'line', 'scatter'), value='bar')

Dropdown(description='Departamento:', options=('ANTIOQUIA', 'ATLANTICO', 'BOGOTA D.C.', 'BOLIVAR', 'BOYACA', '…

SelectMultiple(description='Municipio:', options=(), value=())

Output()

Output()

In [8]:
#V. 0.1

import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Mostrar los campos disponibles
fields = data.columns.tolist()
print("Campos disponibles:")
print(fields)

# Crear los dropdown menus para seleccionar los campos
index_dropdown = widgets.Dropdown(options=fields, description="Index:")
variable_dropdown = widgets.Dropdown(options=fields, description="Variable:")
x_axis_dropdown = widgets.Dropdown(options=fields, description="Eje X:")
y_axis_dropdown = widgets.Dropdown(options=fields, description="Eje Y:")
chart_type_dropdown = widgets.Dropdown(options=["bar", "line", "scatter"], description="Tipo de gráfico:")

# Filtrar el campo "DEPARTAMENTO" para obtener las opciones del dropdown de MUNICIPIO
departamento_options = data["DEPARTAMENTO"].unique()

# Crear el dropdown menu para seleccionar el campo "DEPARTAMENTO"
departamento_dropdown = widgets.Dropdown(options=departamento_options, description="Departamento:")

# Crear el dropdown menu para seleccionar el campo "MUNICIPIO" basado en el filtro del campo "DEPARTAMENTO"
municipio_dropdown = widgets.SelectMultiple(description="Municipio:")

# Función para actualizar las opciones del dropdown de MUNICIPIO basado en el filtro del dropdown de DEPARTAMENTO
def update_municipio_options(change):
    selected_departamentos = [departamento_dropdown.value]
    municipio_options = data[data["DEPARTAMENTO"].isin(selected_departamentos)]["MUNICIPIO"].unique()
    municipio_dropdown.options = municipio_options

# Asignar la función de actualización de opciones al evento "value" del dropdown de DEPARTAMENTO
departamento_dropdown.observe(update_municipio_options, 'value')

# Crear las salidas HTML para la tabla y el gráfico
table_output = widgets.Output()
chart_output = widgets.Output()

# Función para generar y mostrar la tabla y el gráfico seleccionados
def generate_output(change):
    # Obtener las selecciones realizadas
    selected_departamentos = [departamento_dropdown.value]
    selected_municipios = list(municipio_dropdown.value)
    
    # Obtener los campos seleccionados
    index_field = index_dropdown.value
    variable_field = variable_dropdown.value
    x_field = x_axis_dropdown.value
    y_field = y_axis_dropdown.value
    chart_type = chart_type_dropdown.value
    
    # Filtrar los datos basado en las selecciones realizadas
    filtered_data = data[data["DEPARTAMENTO"].isin(selected_departamentos) & data["MUNICIPIO"].isin(selected_municipios)]
    
    # Limpiar las salidas HTML
    table_output.clear_output()
    chart_output.clear_output()
    
    # Generar y mostrar la tabla
    with table_output:
        display(HTML("<h2>Tabla</h2>"))
        display(filtered_data)
    
    # Generar y mostrar el gráfico
    with chart_output:
        try:
            fig = go.Figure()
            
            if chart_type == "bar":
                fig.add_trace(go.Bar(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "line":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="lines",
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "scatter":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="markers",
                    name=f"{y_field} ({x_field})"
                ))

            fig.update_layout(
                title=f"Gráfico de {chart_type} de {y_field} por {x_field}",
                xaxis=dict(title=x_field),
                yaxis=dict(title=y_field)
            )

            fig.show()
        except ValueError as e:
            print(f"No se pudo generar la gráfica. Error: {str(e)}")

# Asignar la función de generación de tabla y gráfico al evento "change" de los dropdown menus
index_dropdown.observe(generate_output, 'value')
variable_dropdown.observe(generate_output, 'value')
x_axis_dropdown.observe(generate_output, 'value')
y_axis_dropdown.observe(generate_output, 'value')
chart_type_dropdown.observe(generate_output, 'value')
departamento_dropdown.observe(generate_output, 'value')
municipio_dropdown.observe(generate_output, 'value')

# Mostrar los dropdown menus y las salidas HTML
display(index_dropdown, variable_dropdown, x_axis_dropdown, y_axis_dropdown, chart_type_dropdown)
display(departamento_dropdown, municipio_dropdown)
display(table_output)
display(chart_output)



Campos disponibles:
['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', 'BUCERROS < 1 AÑO', 'HEMBRAS 1 - 2 AÑOS', 'MACHOS 1 - 2 AÑOS', 'HEMBRAS 2 - 3 AÑOS', 'MACHOS 2 - 3 AÑOS', 'HEMBRAS > 3 AÑOS', 'MACHOS > 3 AÑOS', 'TOTAL BUFALOS - 2022', ' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100', ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS', 'TOTAL FINCAS CON BUFALOS - 2022', 'TERNERAS < 1 AÑO', 'TERNEROS < 1 AÑO', 'TOTAL BOVINOS', 'TOTAL FINCAS CON BOVINOS']


Dropdown(description='Index:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', '…

Dropdown(description='Variable:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO'…

Dropdown(description='Eje X:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', '…

Dropdown(description='Eje Y:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'BUCERRAS < 1 AÑO', '…

Dropdown(description='Tipo de gráfico:', options=('bar', 'line', 'scatter'), value='bar')

Dropdown(description='Departamento:', options=('ANTIOQUIA', 'ATLANTICO', 'BOGOTA D.C.', 'BOLIVAR', 'BOYACA', '…

SelectMultiple(description='Municipio:', options=(), value=())

Output()

Output()

# V. 0.2 Se agrega botón de exportación